#  Sezione 1 – Titolo e Obiettivi della Lezione

## Lezione 4 — Analisi Meteo Avanzata: Quantili, Categorizzazione e GroupBy Multi-Colonna

###  Obiettivi di apprendimento

Al termine di questa lezione sarai in grado di:

1. **Usare quantili** per definire soglie dinamiche basate sui dati stessi
2. **Creare colonne booleane (flag)** per classificazioni binarie rapide
3. **Categorizzare variabili continue** in classi discrete significative
4. **Applicare `.groupby()` multi-colonna** per analisi di combinazioni di fattori
5. **Usare `.transform()`** per aggiungere statistiche di gruppo a ogni riga
6. **Confrontare valori individuali** con medie di gruppo per analisi intra-gruppo

---

###  Prerequisiti

| Prerequisito | Lezione | Concetti da padroneggiare |
|--------------|---------|---------------------------|
| Boolean Indexing | Lesson 02 | Maschere booleane, operatori `&` `|` |
| Feature Engineering | Lesson 02 | Creazione nuove colonne, `.loc[]` |
| GroupBy base | Lesson 03 | Split-Apply-Combine, `.transform()` |

---

###  Perché questa lezione è cruciale?

L'analisi per sottogruppi è il cuore dell'**Exploratory Data Analysis (EDA)** professionale:

| Scenario | Applicazione della lezione |
|----------|---------------------------|
| **Marketing** | Segmentare clienti per fascia di spesa E regione |
| **Finance** | Analizzare rischio per settore E rating |
| **Healthcare** | Confrontare outcome per trattamento E gruppo età |
| **Weather Analysis** | Correlazioni tra pioggia, vento e temperatura |

Il **groupby multi-colonna** rivela pattern **nascosti** nelle interazioni tra variabili!

---

###  Struttura della lezione (8 sezioni)

1. **Titolo e Obiettivi** ← Sei qui
2. **Teoria concettuale profonda** (quantili, categorizzazione, multi-groupby)
3. **Schema mentale / Mappa decisionale**
4. **Sezione dimostrativa** con micro-checkpoint
5. **Reference e Esercizi** + Errori comuni / Debug
6. **Conclusione operativa**
7. **Checklist di fine lezione**
8. **Changelog didattico**

---

###  Librerie utilizzate

```python
import pandas as pd   # Manipolazione dati tabulari
import numpy as np    # Operazioni numeriche (usato per categorizzazione avanzata)
```

---

# 📌 Sezione 2 – Teoria Concettuale Profonda

## 2.1 Quantili e Percentili: Soglie Intelligenti

I **quantili** dividono i dati ordinati in parti uguali. Sono preferibili alle soglie fisse perché:
- Si adattano automaticamente alla distribuzione dei dati
- Sono indipendenti dalla scala delle misurazioni
- Permettono confronti equi tra dataset diversi

### Nomenclatura standard

| Quantile | Notazione | Percentile | Significato |
|----------|-----------|------------|-------------|
| Minimo | Q0 | 0° | Il valore più piccolo |
| **Q1** | $Q_{0.25}$ | **25°** | Il 25% dei dati sta sotto |
| **Q2 (Mediana)** | $Q_{0.5}$ | **50°** | Il 50% dei dati sta sotto |
| **Q3** | $Q_{0.75}$ | **75°** | Il 75% dei dati sta sotto |
| Massimo | Q4 | 100° | Il valore più grande |

### Formula per il percentile p

$$Q_p = \text{valore tale che } p\% \text{ dei dati } \leq Q_p$$

### Interquartile Range (IQR)

L'**IQR** è una misura di dispersione robusta agli outlier:

$$IQR = Q_3 - Q_1$$

Spesso usato per identificare outlier: valori oltre $Q_1 - 1.5 \cdot IQR$ o $Q_3 + 1.5 \cdot IQR$.

---

## 2.2 Categorizzazione di Variabili Continue

Trasformare numeri in categorie (discretizzazione/binning) offre vantaggi:

| Vantaggio | Esempio |
|-----------|---------|
| **Interpretabilità** | "vento alto" vs "vento 17.3 km/h" |
| **Raggruppamenti** | Analisi per classe di intensità |
| **Robustezza** | Meno sensibile a valori esatti |
| **Visualizzazione** | Grafici a barre più chiari |

### Approcci alla categorizzazione

| Approccio | Descrizione | Quando usarlo |
|-----------|-------------|---------------|
| **Soglie fisse** | Definite da conoscenza del dominio | Hai standard di settore |
| **Soglie da quantili** | Basate su percentili dei dati | Vuoi gruppi di uguale numerosità |
| **`pd.cut()`** | Binning con intervalli uguali | Vuoi range di uguali ampiezze |
| **`pd.qcut()`** | Binning con quantili | Vuoi gruppi di uguale numerosità |

---

## 2.3 GroupBy Multi-Colonna

Raggruppare per **combinazioni** di più variabili rivela pattern nascosti:

```
           GroupBy SINGOLA COLONNA              GroupBy MULTI-COLONNA
                    |                                    |
                    v                                    v
         ┌─────────────────┐                 ┌─────────────────────────┐
         │ Gruppi:          │                │ Gruppi (combinazioni):    │
         │  - Pioggia=0     │                │  - (Pioggia=0, Vento=Basso) │
         │  - Pioggia=1     │                │  - (Pioggia=0, Vento=Medio) │
         └─────────────────┘                │  - (Pioggia=0, Vento=Alto)  │
                                            │  - (Pioggia=1, Vento=Basso) │
                                            │  - (Pioggia=1, Vento=Medio) │
                                            │  - (Pioggia=1, Vento=Alto)  │
                                            └─────────────────────────┘
```

### Sintassi

```python
# Singola colonna
df.groupby("Pioggia")["Temperatura"].mean()

# Multi-colonna (lista di colonne)
df.groupby(["Pioggia", "Vento_Class"])["Temperatura"].mean()
```

### MultiIndex nel risultato

Il risultato di un groupby multi-colonna ha un **MultiIndex** (indice gerarchico):

```
Pioggia  Vento_Class
0        Basso          21.0
         Medio          21.0
1        Alto           18.5
         Medio          20.0
Name: Temperatura, dtype: float64
```

---

## 2.4 Transform in Contesto Multi-Gruppo

Con groupby multi-colonna, `transform()` diventa ancora più potente:

```python
# Media per combinazione Pioggia × Vento_Class, assegnata a ogni riga
df["Media_Combo"] = df.groupby(["Pioggia", "Vento_Class"])["Temp"].transform("mean")
```

Ogni riga riceve la media del suo **specifico sottogruppo**, non la media globale o di un singolo fattore.

In [ ]:
# =============================================================================
# SETUP: Import librerie e creazione DataFrame meteo
# =============================================================================
# In questa lezione esploreremo un dataset meteo settimanale.
# Useremo 5 variabili per dimostrare:
# - Quantili per soglie dinamiche
# - Categorizzazione di variabili continue
# - GroupBy multi-colonna
# - Transform per analisi intra-gruppo

import pandas as pd   # Libreria per manipolazione dati tabulari
import numpy as np    # Libreria per operazioni numeriche

# Creiamo un DataFrame con dati meteorologici settimanali
df = pd.DataFrame({
    "Giorno": ["Lun", "Mar", "Mer", "Gio", "Ven", "Sab", "Dom"],
    "Temperatura": [18, 20, 21, 19, 23, 25, 22],  # Gradi Celsius
    "Umidita": [60, 55, 58, 63, 50, 45, 52],      # Percentuale
    "Pioggia": [0, 1, 0, 1, 0, 0, 1],             # 0=No, 1=Sì
    "Vento": [5, 12, 8, 20, 7, 6, 15]             # km/h
})

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifica struttura iniziale
# -----------------------------------------------------------------------------
print("=== SETUP COMPLETATO ===\n")
print(f"DataFrame meteo creato con successo!")
print(f"Dimensioni: {df.shape[0]} righe × {df.shape[1]} colonne")
print(f"Colonne: {list(df.columns)}")

# Statistiche rapide
print(f"\n=== Statistiche rapide ===")
print(f"Temperatura: min={df['Temperatura'].min()}°C, max={df['Temperatura'].max()}°C")
print(f"Umidità: min={df['Umidita'].min()}%, max={df['Umidita'].max()}%")
print(f"Vento: min={df['Vento'].min()} km/h, max={df['Vento'].max()} km/h")
print(f"Giorni con pioggia: {df['Pioggia'].sum()}/7")

# Verifica tipi di dato
assert df["Temperatura"].dtype in ['int64', 'float64'], "Temperatura deve essere numerica!"
assert df["Pioggia"].isin([0, 1]).all(), "Pioggia deve contenere solo 0 e 1!"
print("\n--- Micro-checkpoint: struttura dati verificata ---")

df

,Giorno,Temperatura,Umidita,Pioggia,Vento
0,Lun,18,60,0,5
1,Mar,20,55,1,12
2,Mer,21,58,0,8
3,Gio,19,63,1,20
4,Ven,23,50,0,7
5,Sab,25,45,0,6
6,Dom,22,52,1,15


---

# 📌 Sezione 3 – Schema Mentale / Mappa Decisionale

## 3.1 Quando Usare Quale Tecnica?

```
╔════════════════════════════════════════════════════════════════════╗
║            DEVO ANALIZZARE DATI PER GRUPPI?                        ║
╚════════════════════════════════════════════════════════════════════╝
                              │
                              ▼
              Voglio statistiche RIASSUNTIVE per gruppo?
                           /            \
                         SÌ              NO
                          │               │
                          ▼               ▼
               .groupby().agg()     Voglio aggiungere stat
               .groupby().mean()    a OGNI RIGA del df?
                          │                    │
                          ▼                    ▼
               Output ridotto           .transform()
              (1 riga/gruppo)        Output stesso size
```

---

## 3.2 Scelta della Soglia: Fissa vs Dinamica

```
╔════════════════════════════════════════════════════════════════════╗
║           COME DEFINIRE LA SOGLIA?                                 ║
╚════════════════════════════════════════════════════════════════════╝
                              │
              ┌───────────────┼───────────────┐
              │               │               │
              ▼               ▼               ▼
         SOGLIA            SOGLIA          SOGLIA
         FISSA            DA MEDIA      DA QUANTILE
              │               │               │
              ▼               ▼               ▼
         Conosci          Vuoi il         Vuoi gruppi
         standard         "sopra/sotto    di numerosità
         di settore?      la media"?      controllata?
              │               │               │
              ▼               ▼               ▼
         > 100°C         > df.mean()    > df.quantile(0.75)
```

---

## 3.3 Pattern di Categorizzazione

```
VARIABILE CONTINUA → CATEGORICA
               │
               ▼
       Conosco le soglie esatte?
              /        \
            SÌ          NO
             │           │
             ▼           ▼
     Maschere +      pd.cut() o
     .loc[]          pd.qcut()
             │           │
             ▼           ▼
     Controllo       Binning
     totale          automatico
```

---

## 3.4 Checklist Pre-Analisi

Prima di scrivere codice, rispondi a queste domande:

| # | Domanda | Impatto sulla scelta |
|---|---------|---------------------|
| 1 | Quante variabili definiscono i gruppi? | Singola o multi-colonna |
| 2 | Quale variabile voglio aggregare? | Selezione con `["colonna"]` |
| 3 | Quale funzione di aggregazione? | mean, sum, count, custom |
| 4 | Mi serve output ridotto o espanso? | agg vs transform |
| 5 | Devo categorizzare prima del groupby? | Preprocessing necessario |

---

## 3.5 Workflow Completo: Analisi Multi-Dimensionale

```
┌─────────────────────────────────────────────────────────────┐
│ 1. CATEGORIZZA variabili continue                           │
│    df["Vento_Class"] = ... (Basso/Medio/Alto)               │
└─────────────────────────────────────────────────────────────┘
                           │
                           ▼
┌─────────────────────────────────────────────────────────────┐
│ 2. RAGGRUPPA per combinazioni                               │
│    df.groupby(["Pioggia", "Vento_Class"])                   │
└─────────────────────────────────────────────────────────────┘
                           │
                           ▼
┌─────────────────────────────────────────────────────────────┐
│ 3. AGGREGA o TRASFORMA                                      │
│    .mean() → ridotto | .transform("mean") → espanso         │
└─────────────────────────────────────────────────────────────┘
                           │
                           ▼
┌─────────────────────────────────────────────────────────────┐
│ 4. CONFRONTA con media di gruppo                            │
│    df["Delta"] = df["Val"] - df["Media_Gruppo"]             │
└─────────────────────────────────────────────────────────────┘
```

---

# 📌 Sezione 4 – Sezione Dimostrativa

## 4.1 Panoramica della Dimostrazione

In questa sezione costruiremo passo passo un'analisi meteo completa:

| Step | Obiettivo | Tecnica |
|------|-----------|---------|
| 4.2 | Definire soglia "umidità alta" | Quantile (Q3) |
| 4.3 | Creare flag binario | Colonna booleana |
| 4.4-4.5 | Categorizzare il vento | Maschere + `.loc[]` |
| 4.6 | Analisi per combinazioni | GroupBy multi-colonna |
| 4.7 | Confronto intra-gruppo | Transform + Delta |

### Obiettivo finale

Arrivare a un DataFrame arricchito con:
- Flag `umido_alto` (True/False)
- Classe `Vento_Class` (Basso/Medio/Alto)
- Media umidità per gruppo `Umidita_Media_Gruppo`
- Scostamento `Diff_da_Media`

---

## 4.2 Calcolo Quantili per Soglie Dinamiche

### Perché usare i quantili invece di soglie fisse?

| Approccio | Pro | Contro |
|-----------|-----|--------|
| **Soglia fissa** (es. 60%) | Facile da comunicare | Non si adatta ai dati |
| **Soglia da quantile** | Si adatta ai dati | Richiede calcolo |

### Il 75° percentile (Q3)

Calcoliamo Q3 dell'umidità: il valore sotto cui sta il **75% dei giorni**.  
I giorni sopra Q3 saranno nel "top 25% più umido" → li classificheremo come "umidità alta".

### Anatomia del comando

```python
df["Umidita"].quantile(0.75)
│     │             │
│     │             └── Frazione (0.75 = 75° percentile)
│     └── Colonna da analizzare
└── DataFrame
```

**Nota importante**: l'argomento è un **decimale tra 0 e 1**, non un intero!
- ✅ Corretto: `.quantile(0.75)`
- ❌ Sbagliato: `.quantile(75)` → restituisce il 75° elemento, non il 75° percentile!

In [ ]:
# =============================================================================
# CALCOLO Q3 (75° PERCENTILE) DELL'UMIDITÀ
# =============================================================================
# Obiettivo: calcolare una soglia dinamica che si adatta ai dati.
# Q3 = valore sotto cui sta il 75% dei dati
# Quindi: valori > Q3 sono nel top 25% (li chiameremo "umidità alta")

print("=== CALCOLO QUANTILE Q3 (75° PERCENTILE) ===\n")

# Calcolo del 75° percentile
q3_umidita = df["Umidita"].quantile(0.75)

print(f"75° percentile (Q3) dell'umidità: {q3_umidita}%")
print(f"\nInterpretazione:")
print(f"  • Il 75% dei giorni ha umidità ≤ {q3_umidita}%")
print(f"  • I giorni con umidità > {q3_umidita}% sono nel top 25% più umido")

# Mostra anche altri quantili per contesto
print(f"\n=== Confronto con altri quantili ===")
print(f"Q1 (25° percentile): {df['Umidita'].quantile(0.25)}%")
print(f"Q2 (mediana):        {df['Umidita'].quantile(0.50)}%")
print(f"Q3 (75° percentile): {q3_umidita}%")
print(f"IQR = Q3 - Q1:       {df['Umidita'].quantile(0.75) - df['Umidita'].quantile(0.25)}%")

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifica che Q3 sia nel range dei dati
# -----------------------------------------------------------------------------
min_umidita = df["Umidita"].min()
max_umidita = df["Umidita"].max()

assert min_umidita <= q3_umidita <= max_umidita, \
    f"ERRORE: Q3 ({q3_umidita}) fuori dal range [{min_umidita}, {max_umidita}]!"

print(f"\n--- Micro-checkpoint: Q3 = {q3_umidita}% nel range [{min_umidita}%, {max_umidita}%] ✓ ---")

np.float64(59.0)

---

## 4.3 Creare una Colonna Booleana (Flag)

### Cos'è un "flag"?

Un **flag** è una colonna binaria (True/False o 0/1) che indica se una condizione è verificata.

| Uso | Esempio |
|-----|---------|
| **Filtraggio rapido** | `df[df["flag"]]` |
| **Conteggio** | `df["flag"].sum()` conta i True |
| **Raggruppamento** | `df.groupby("flag")` divide in 2 gruppi |
| **Feature per ML** | Modelli usano flag come input binario |

### Pattern di creazione

```python
# Sintassi base: confronto → booleano
df["flag"] = df["colonna"] > soglia

# Il risultato è una Series di True/False
```

### Interpretazione

| Valore | Significato nel nostro caso |
|--------|----------------------------|
| `True` | Umidità > Q3 (top 25% più umido) |
| `False` | Umidità ≤ Q3 (75% normale) |

### Conversione a intero (opzionale)

Se preferisci 0/1 invece di True/False:
```python
df["flag_int"] = (df["colonna"] > soglia).astype(int)
```

In [ ]:
# =============================================================================
# CREAZIONE COLONNA BOOLEANA (FLAG)
# =============================================================================
# Obiettivo: creare una colonna "umido_alto" che vale True quando
# l'umidità supera il 75° percentile (Q3).
#
# Questo flag ci permetterà di:
# - Filtrare rapidamente i giorni umidi
# - Contare quanti giorni sono "umidi alti"
# - Usare come variabile di raggruppamento

print("=== CREAZIONE FLAG 'umido_alto' ===\n")

# Creazione del flag booleano
# La condizione (df["Umidita"] > q3_umidita) restituisce True/False per ogni riga
df["umido_alto"] = df["Umidita"] > q3_umidita

print(f"Soglia usata: {q3_umidita}%")
print(f"Nuova colonna 'umido_alto' creata!")

# Visualizza la distribuzione
n_umidi_alti = df["umido_alto"].sum()  # sum() conta i True
n_totali = len(df)
percentuale = n_umidi_alti / n_totali * 100

print(f"\n=== Distribuzione ===")
print(f"Giorni con umidità > Q3: {n_umidi_alti}/{n_totali} ({percentuale:.1f}%)")
print(f"Giorni con umidità ≤ Q3: {n_totali - n_umidi_alti}/{n_totali} ({100 - percentuale:.1f}%)")

# Mostra quali giorni sono classificati come "umidi alti"
print(f"\n=== Giorni classificati 'umido_alto' ===")
print(df.loc[df["umido_alto"], ["Giorno", "Umidita", "umido_alto"]])

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifica coerenza del flag
# -----------------------------------------------------------------------------
# Tutti i giorni con umido_alto=True devono avere Umidita > q3_umidita
if n_umidi_alti > 0:
    min_umidita_alti = df.loc[df["umido_alto"], "Umidita"].min()
    assert min_umidita_alti > q3_umidita, \
        f"ERRORE: trovato umido_alto con umidità {min_umidita_alti} ≤ soglia {q3_umidita}!"
    print(f"\n--- Micro-checkpoint: min umidità nei 'umido_alto' = {min_umidita_alti}% > {q3_umidita}% ✓ ---")

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto
0,Lun,18,60,0,5,True
1,Mar,20,55,1,12,False
2,Mer,21,58,0,8,False
3,Gio,19,63,1,20,True
4,Ven,23,50,0,7,False
5,Sab,25,45,0,6,False
6,Dom,22,52,1,15,False


---

## 4.4 Categorizzare una Variabile Continua

### Perché categorizzare il vento?

Il vento (in km/h) è una variabile continua. Per analisi per gruppi, conviene trasformarlo in categorie:

| Categoria | Range | Interpretazione |
|-----------|-------|-----------------|
| **Basso** | < 8 km/h | Brezza leggera |
| **Medio** | 8-14 km/h | Vento moderato |
| **Alto** | ≥ 15 km/h | Vento forte |

### Approccio: Maschere booleane + `.loc[]`

Il metodo è in due step:

1. **Creare maschere booleane** per ogni categoria
2. **Assegnare etichette** usando `.loc[maschera, colonna] = valore`

### Perché inizializzare prima la colonna?

Se usi `.loc[]` senza inizializzare, le righe non coperte restano `NaN`:

```python
# ❌ PROBLEMA: righe non coperte diventano NaN
df.loc[cond1, "Cat"] = "A"
df.loc[cond2, "Cat"] = "B"
# Le righe in cond3 sono NaN!

# ✅ SOLUZIONE: inizializza con default
df["Cat"] = "C"  # Default per cond3
df.loc[cond1, "Cat"] = "A"
df.loc[cond2, "Cat"] = "B"
```

### Verifica fondamentale

Ogni riga deve appartenere a **esattamente una** categoria.  
Verifica: `somma delle maschere == numero righe`

In [ ]:
# =============================================================================
# STEP 1: Creare le maschere booleane per ogni categoria
# =============================================================================
# Definiamo le condizioni per classificare il vento:
# - Basso: < 8 km/h
# - Medio: 8-14 km/h (incluso 8, escluso 15)
# - Alto: >= 15 km/h

print("=== CATEGORIZZAZIONE VENTO: Step 1 - Maschere ===\n")

# Definiamo le soglie (costanti per chiarezza)
SOGLIA_MEDIO = 8
SOGLIA_ALTO = 15

# Creiamo le maschere booleane
vento_basso = df["Vento"] < SOGLIA_MEDIO
vento_medio = (df["Vento"] >= SOGLIA_MEDIO) & (df["Vento"] < SOGLIA_ALTO)
vento_alto = df["Vento"] >= SOGLIA_ALTO

# Visualizza le maschere
print(f"Soglie: Basso < {SOGLIA_MEDIO}, Medio {SOGLIA_MEDIO}-{SOGLIA_ALTO-1}, Alto >= {SOGLIA_ALTO}")
print(f"\nMaschere booleane create:")
print(f"  Vento Basso (<{SOGLIA_MEDIO}):      {list(vento_basso.values)}")
print(f"  Vento Medio ({SOGLIA_MEDIO}-{SOGLIA_ALTO-1}):    {list(vento_medio.values)}")
print(f"  Vento Alto (>={SOGLIA_ALTO}):    {list(vento_alto.values)}")

# Conta per categoria
print(f"\nDistribuzione:")
print(f"  Basso: {vento_basso.sum()} giorni")
print(f"  Medio: {vento_medio.sum()} giorni")
print(f"  Alto:  {vento_alto.sum()} giorni")

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Ogni riga deve appartenere a ESATTAMENTE una categoria
# -----------------------------------------------------------------------------
totale_classificati = vento_basso.sum() + vento_medio.sum() + vento_alto.sum()
n_righe = len(df)

assert totale_classificati == n_righe, \
    f"ERRORE: {totale_classificati} classificati su {n_righe} righe!"

# Verifica che nessuna riga appartenga a più categorie
doppi = ((vento_basso & vento_medio) | (vento_medio & vento_alto) | (vento_basso & vento_alto)).sum()
assert doppi == 0, f"ERRORE: {doppi} righe appartengono a più categorie!"

print(f"\n--- Micro-checkpoint: tutte le {n_righe} righe classificate univocamente ✓ ---")

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto
0,Lun,18,60,0,5,True
1,Mar,20,55,1,12,False
2,Mer,21,58,0,8,False
3,Gio,19,63,1,20,True
4,Ven,23,50,0,7,False


---

## 4.5 Assegnare Etichette con `.loc[]`

### Il pattern sicuro

```python
# 1. Inizializza con il valore più comune o default
df["Categoria"] = "Default"

# 2. Sovrascrivi per le categorie specifiche
df.loc[condizione_1, "Categoria"] = "Valore_1"
df.loc[condizione_2, "Categoria"] = "Valore_2"
```

### Perché questo ordine?

- Inizializzando prima, **nessuna riga resta NaN**
- L'ordine delle sovrascritture conta: l'ultima vince
- Se le condizioni sono mutuamente esclusive, l'ordine non importa

### Alternativa: `np.select()` (più compatto)

```python
condizioni = [vento_basso, vento_medio, vento_alto]
valori = ["Basso", "Medio", "Alto"]
df["Vento_Class"] = np.select(condizioni, valori, default="Sconosciuto")
```

Useremo il metodo `.loc[]` per chiarezza didattica.

In [ ]:
# =============================================================================
# STEP 2: Assegnare le etichette usando .loc[]
# =============================================================================
# Strategia: 
# 1. Inizializziamo con "Basso" (la categoria più numerosa come default)
# 2. Sovrascriviamo con .loc[] per le altre categorie

print("=== CATEGORIZZAZIONE VENTO: Step 2 - Etichette ===\n")

# Step 2a: Inizializza con il valore di default
df["Vento_Class"] = "Basso"
print("Colonna 'Vento_Class' inizializzata con default 'Basso'")

# Step 2b: Sovrascrivi per le categorie Medio e Alto
df.loc[vento_medio, "Vento_Class"] = "Medio"
df.loc[vento_alto, "Vento_Class"] = "Alto"
print("Valori sovrascritti per categorie 'Medio' e 'Alto'")

# Mostra la distribuzione risultante
print(f"\n=== Distribuzione finale ===")
print(df["Vento_Class"].value_counts())

# Mostra mapping vento → categoria
print(f"\n=== Mapping Vento → Categoria ===")
print(df[["Giorno", "Vento", "Vento_Class"]].to_string(index=False))

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifica coerenza della categorizzazione
# -----------------------------------------------------------------------------
print("\n--- Micro-checkpoint: Verifica coerenza ---")

# Verifica categoria "Alto": tutti devono avere Vento >= 15
if df["Vento_Class"].eq("Alto").any():
    min_vento_alto = df.loc[df["Vento_Class"] == "Alto", "Vento"].min()
    assert min_vento_alto >= SOGLIA_ALTO, \
        f"ERRORE: categoria 'Alto' con vento {min_vento_alto} < {SOGLIA_ALTO}!"
    print(f"  Alto: min vento = {min_vento_alto} km/h >= {SOGLIA_ALTO} ✓")

# Verifica categoria "Basso": tutti devono avere Vento < 8
if df["Vento_Class"].eq("Basso").any():
    max_vento_basso = df.loc[df["Vento_Class"] == "Basso", "Vento"].max()
    assert max_vento_basso < SOGLIA_MEDIO, \
        f"ERRORE: categoria 'Basso' con vento {max_vento_basso} >= {SOGLIA_MEDIO}!"
    print(f"  Basso: max vento = {max_vento_basso} km/h < {SOGLIA_MEDIO} ✓")

# Nessun NaN nella nuova colonna
assert df["Vento_Class"].notna().all(), "ERRORE: trovati NaN nella categorizzazione!"
print(f"  Nessun valore NaN ✓")

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto,Vento_Class
0,Lun,18,60,0,5,True,Basso
1,Mar,20,55,1,12,False,Medio
2,Mer,21,58,0,8,False,Medio
3,Gio,19,63,1,20,True,Alto
4,Ven,23,50,0,7,False,Basso
5,Sab,25,45,0,6,False,Basso
6,Dom,22,52,1,15,False,Alto


---

## 4.6 GroupBy Multi-Colonna per Aggregazioni

### Perché usare più colonne per raggruppare?

| Singolo fattore | Domanda |
|-----------------|---------|
| Solo Pioggia | "Qual è la media quando piove vs non piove?" |
| Solo Vento | "Qual è la media con vento basso/medio/alto?" |

| **Due fattori** | **Domanda più ricca** |
|-----------------|----------------------|
| Pioggia × Vento | "Come cambia la media per ogni combinazione di pioggia E vento?" |

Questo rivela **interazioni** tra fattori che l'analisi singola non mostra!

### Sintassi

```python
# Lista di colonne tra parentesi quadre
df.groupby(["Pioggia", "Vento_Class"])[["Temperatura", "Umidita"]].mean()
```

### Il risultato: MultiIndex

Il DataFrame risultante ha un **indice gerarchico** (MultiIndex):
- Livello 0: Pioggia (0 o 1)
- Livello 1: Vento_Class (Basso, Medio, Alto)

Per accedere ai dati:
```python
risultato.loc[(0, "Basso")]  # Pioggia=0 E Vento=Basso
```

In [ ]:
# =============================================================================
# GROUPBY MULTI-COLONNA: Aggregazioni per combinazioni
# =============================================================================
# Obiettivo: calcolare la media di Temperatura e Umidità per ogni
# combinazione di (Pioggia, Vento_Class).
#
# Questo ci mostrerà, ad esempio:
# - Media temperatura nei giorni senza pioggia E con vento basso
# - Media umidità nei giorni con pioggia E vento alto
# - ecc.

print("=== GROUPBY MULTI-COLONNA ===\n")

# Raggruppiamo per due colonne: Pioggia (0/1) e Vento_Class (Basso/Medio/Alto)
# Selezioniamo le colonne da aggregare e calcoliamo la media
df_grouped = df.groupby(["Pioggia", "Vento_Class"])[["Temperatura", "Umidita"]].mean()

print(f"Numero di gruppi (combinazioni): {len(df_grouped)}")
print(f"\nMedia di Temperatura e Umidità per ogni combinazione:")
print(df_grouped)

# Analisi dei gruppi
print(f"\n=== Analisi dei gruppi ===")
print(f"Tipo indice: {type(df_grouped.index).__name__}")
print(f"Livelli indice: {df_grouped.index.names}")

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifica struttura MultiIndex
# -----------------------------------------------------------------------------
assert isinstance(df_grouped.index, pd.MultiIndex), \
    "ERRORE: il risultato dovrebbe avere un MultiIndex!"

# Verifica che il numero di gruppi sia <= combinazioni possibili
n_pioggia = df["Pioggia"].nunique()  # 2: 0 e 1
n_vento = df["Vento_Class"].nunique()  # 3: Basso, Medio, Alto
max_gruppi = n_pioggia * n_vento  # 6 combinazioni possibili

assert len(df_grouped) <= max_gruppi, \
    f"ERRORE: {len(df_grouped)} gruppi > {max_gruppi} combinazioni possibili!"

print(f"\n--- Micro-checkpoint ---")
print(f"  MultiIndex creato correttamente ✓")
print(f"  Gruppi trovati: {len(df_grouped)}/{max_gruppi} combinazioni possibili ✓")
print(f"  (alcuni gruppi potrebbero mancare se non ci sono dati per quella combinazione)")

Temperatura    Umidita
Pioggia Vento_Class                        
0       Basso               22.0  51.666667
        Medio               21.0  58.000000
1       Alto                20.5  57.500000
        Medio               20.0  55.000000

---

## 4.7 Transform: Aggiungere Statistiche di Gruppo a Ogni Riga

### Il problema che transform risolve

Vogliamo confrontare ogni giorno con la media del suo **specifico sottogruppo**  
(combinazione Pioggia × Vento_Class), non con la media globale.

### La differenza cruciale

| Metodo | Output | Uso |
|--------|--------|-----|
| `.mean()` | 1 valore per gruppo (ridotto) | Report, statistiche |
| `.transform("mean")` | N valori (stessa lunghezza) | Aggiungere a ogni riga |

### Cosa produce transform con multi-groupby?

Ogni riga riceve la media del suo **specifico sottogruppo**:

| Giorno | Pioggia | Vento_Class | Umidita | Umidita_Media_Gruppo |
|--------|---------|-------------|---------|----------------------|
| Lun | 0 | Basso | 60 | Media di (0, Basso) |
| Mar | 1 | Medio | 55 | Media di (1, Medio) |

### Il pattern completo

```python
# 1. Calcola media per combinazione e assegna a ogni riga
df["Media_Gruppo"] = df.groupby(["Col1", "Col2"])["Val"].transform("mean")

# 2. Calcola lo scostamento (delta)
df["Delta"] = df["Val"] - df["Media_Gruppo"]
```

Questo permette di identificare giorni **anomali** rispetto ai giorni simili.

In [ ]:
# =============================================================================
# TRANSFORM: Media di gruppo assegnata a ogni riga
# =============================================================================
# Obiettivo: per ogni riga, aggiungere la media dell'umidità del suo
# specifico sottogruppo (combinazione Pioggia × Vento_Class).
#
# Questo ci permette poi di calcolare quanto ogni giorno si discosta
# dalla media dei giorni "simili" (stesso tipo di pioggia e vento).

print("=== TRANSFORM CON GROUPBY MULTI-COLONNA ===\n")

# Calcola la media per ogni combinazione (Pioggia, Vento_Class)
# e assegna a ogni riga la media del suo gruppo
df["Umidita_Media_Gruppo"] = df.groupby(["Pioggia", "Vento_Class"])["Umidita"].transform("mean")

print("Colonna 'Umidita_Media_Gruppo' creata!")
print(f"Ogni riga ora ha la media umidità del suo sottogruppo.")

# -----------------------------------------------------------------------------
# MICRO-CHECKPOINT: Verifica lunghezza invariata
# -----------------------------------------------------------------------------
assert len(df["Umidita_Media_Gruppo"]) == len(df), \
    "ERRORE: transform deve mantenere la stessa lunghezza!"
print(f"\n--- Micro-checkpoint: lunghezza mantenuta ({len(df)} righe) ✓ ---")

# Calcola lo scostamento (delta) dalla media del gruppo
df["Diff_da_Media"] = df["Umidita"] - df["Umidita_Media_Gruppo"]

print("\n=== CALCOLO SCOSTAMENTO (DELTA) ===")
print("Colonna 'Diff_da_Media' creata!")
print("\nInterpretazione:")
print("  • Diff > 0: umidità SOPRA la media del gruppo")
print("  • Diff < 0: umidità SOTTO la media del gruppo")
print("  • Diff = 0: esattamente nella media del gruppo")

# Mostra il DataFrame finale
print("\n=== DataFrame finale con analisi di gruppo ===")
print(df[["Giorno", "Pioggia", "Vento_Class", "Umidita", 
         "Umidita_Media_Gruppo", "Diff_da_Media"]].to_string(index=False))

# Identifica il giorno più anomalo (maggiore scostamento in valore assoluto)
idx_max_delta = df["Diff_da_Media"].abs().idxmax()
giorno_anomalo = df.loc[idx_max_delta, "Giorno"]
delta_anomalo = df.loc[idx_max_delta, "Diff_da_Media"]

print(f"\n=== Giorno più anomalo ===")
print(f"  {giorno_anomalo}: scostamento di {delta_anomalo:+.1f}% dalla media del suo gruppo")

df

,Giorno,Temperatura,Umidita,Pioggia,Vento,umido_alto,Vento_Class,Umidita_Vento
0,Lun,18,60,0,5,True,Basso,51.666667
1,Mar,20,55,1,12,False,Medio,55.000000
2,Mer,21,58,0,8,False,Medio,58.000000
3,Gio,19,63,1,20,True,Alto,57.500000
4,Ven,23,50,0,7,False,Basso,51.666667
5,Sab,25,45,0,6,False,Basso,51.666667
6,Dom,22,52,1,15,False,Alto,57.500000


---

# 5) Esercizi risolti (step by step)

## Statistiche Descrittive

| Metodo | Sintassi | Output |
|--------|----------|--------|
| `.quantile(q)` | `df["Col"].quantile(0.75)` | Valore al percentile q |
| `.mean()` | `df["Col"].mean()` | Media aritmetica |
| `.median()` | `df["Col"].median()` | Mediana (50° percentile) |
| `.std()` | `df["Col"].std()` | Deviazione standard |

## Creazione Colonne

| Pattern | Esempio | Risultato |
|---------|---------|-----------|
| Booleana | `df["Flag"] = df["A"] > soglia` | True/False |
| Con `.loc[]` | `df.loc[mask, "Cat"] = "Val"` | Assegnazione condizionale |
| Aritmetica | `df["C"] = df["A"] - df["B"]` | Colonna calcolata |

## GroupBy e Aggregazioni

| Metodo | Sintassi | Output |
|--------|----------|--------|
| `.groupby()` | `df.groupby("Col")` | Oggetto GroupBy |
| `.mean()` | `df.groupby("A")["B"].mean()` | Media per gruppo (ridotto) |
| `.sum()` | `df.groupby("A")["B"].sum()` | Somma per gruppo |
| `.count()` | `df.groupby("A")["B"].count()` | Conteggio per gruppo |
| `.agg()` | `df.groupby("A").agg({"B": "mean", "C": "sum"})` | Aggregazioni multiple |

## Transform

| Metodo | Sintassi | Output |
|--------|----------|--------|
| `.transform("mean")` | `df.groupby("A")["B"].transform("mean")` | Media gruppo per ogni riga |
| `.transform("sum")` | `df.groupby("A")["B"].transform("sum")` | Somma gruppo per ogni riga |
| `.transform(func)` | `df.groupby("A")["B"].transform(lambda x: x - x.mean())` | Funzione custom |

## GroupBy Multi-Colonna

| Sintassi | Cosa Fa |
|----------|---------|
| `df.groupby(["A", "B"])` | Gruppi per combinazioni di A e B |
| `df.groupby(["A", "B"])["C"].mean()` | Media di C per ogni combinazione |
| Risultato | MultiIndex con (A, B) come indice |

---

## Glossario essenziale (dentro la sezione 2)
Questa tabella rimane parte della teoria: definisce i termini usati e ti evita ambiguita' quando applichi i passi pratici.

| Termine | Definizione |
|---------|-------------|
| **Quantile** | Valore che divide i dati ordinati in parti proporzionali |
| **Percentile** | Quantile espresso in percentuale (es. 75° percentile = Q3) |
| **Q1, Q2, Q3** | Primo, secondo (mediana), terzo quartile |
| **IQR** | Interquartile Range = Q3 - Q1, misura di dispersione |
| **Categorizzazione** | Trasformazione di variabile continua in classi discrete |
| **GroupBy** | Operazione che raggruppa righe per valori di una/piu colonne |
| **Split-Apply-Combine** | Paradigma: dividi per gruppi, applica funzione, combina risultati |
| **Aggregazione** | Riduzione di piu valori a uno solo (media, somma, conteggio) |
| **Transform** | Applicazione di funzione che mantiene la lunghezza originale |
| **MultiIndex** | Indice gerarchico con piu livelli (risultato di groupby multi-colonna) |
| **Flag** | Colonna booleana usata per classificazione binaria |
| **Soglia Dinamica** | Soglia calcolata dai dati (es. media, quantile) vs. fissa |
| **Feature Derivata** | Nuova colonna creata da trasformazioni di colonne esistenti |

---

## Errori comuni e debug rapido (dentro la sezione 5)
Questi errori completano gli esercizi: leggi sintomi, causa probabile e fix prima di cambiare codice.

## Errore 1: Confondere `.mean()` con `.transform("mean")`

```python
## SBAGLIATO - riduce a un valore per gruppo
df["Media"] = df.groupby("Cat")["Val"].mean()  # ValueError: lunghezza diversa!

## CORRETTO - mantiene la lunghezza
df["Media"] = df.groupby("Cat")["Val"].transform("mean")
```

## Errore 2: Dimenticare le Parentesi in Condizioni Multiple

```python
## SBAGLIATO
vento_medio = df["Vento"] >= 8 & df["Vento"] < 15  # Errore precedenza!

## CORRETTO
vento_medio = (df["Vento"] >= 8) & (df["Vento"] < 15)
```

## Errore 3: Usare `.loc[]` Senza Colonna

```python
## SBAGLIATO - assegna a TUTTE le colonne!
df.loc[mask] = "Valore"

## CORRETTO - specifica la colonna
df.loc[mask, "Colonna"] = "Valore"
```

## Errore 4: Non Inizializzare la Colonna Prima di `.loc[]`

```python
## SBAGLIATO - colonna non esiste ancora
df.loc[cond1, "Cat"] = "A"
df.loc[cond2, "Cat"] = "B"  # Le righe non in cond1 o cond2 saranno NaN!

## CORRETTO - inizializza con default
df["Cat"] = "Default"
df.loc[cond1, "Cat"] = "A"
df.loc[cond2, "Cat"] = "B"
```

## Errore 5: Quantile Fuori Range [0, 1]

```python
## SBAGLIATO - percentile come intero
df["Col"].quantile(75)  # Errore o risultato errato!

## CORRETTO - decimale tra 0 e 1
df["Col"].quantile(0.75)
```

## Errore 6: GroupBy su Colonna Inesistente

```python
## SBAGLIATO - typo nel nome
df.groupby("Categria")["Val"].mean()  # KeyError!

## CORRETTO - verifica i nomi prima
print(df.columns)
df.groupby("Categoria")["Val"].mean()
```

## Errore 7: Aspettarsi Modifica In-Place

```python
## ATTENZIONE - groupby().mean() restituisce NUOVO oggetto
df.groupby("Cat")["Val"].mean()  # df NON cambia!

## Per usare il risultato, salvalo
result = df.groupby("Cat")["Val"].mean()
```

---

# 📌 Sezione 6 – Conclusione Operativa

## 6.1 Cosa Hai Imparato

In questa lezione hai acquisito competenze avanzate per l'analisi dei dati:

| Competenza | Dettaglio |
|------------|-----------|
| **Quantili come soglie** | Definire "alto/basso" basandosi sui dati stessi |
| **Colonne booleane (flag)** | Classificazione binaria rapida |
| **Categorizzazione** | Trasformare numeri in classi significative |
| **GroupBy multi-colonna** | Analizzare combinazioni di fattori |
| **Transform multi-gruppo** | Statistiche per sottogruppo a ogni riga |
| **Confronti intra-gruppo** | Quanto sopra/sotto la media del proprio gruppo |

---

## 6.2 Pattern Ricorrente: Analisi Multi-Dimensionale

Questo è il workflow completo per analisi avanzate:

```python
# 1. CATEGORIZZA variabili continue
df["Cat"] = "Default"
df.loc[condizione_1, "Cat"] = "Valore_1"
df.loc[condizione_2, "Cat"] = "Valore_2"

# 2. CREA FLAG booleani per condizioni importanti
df["Flag"] = df["Colonna"] > df["Colonna"].quantile(0.75)

# 3. RAGGRUPPA per combinazioni di fattori
df.groupby(["Fattore1", "Fattore2"])["Valore"].mean()

# 4. AGGIUNGI statistica a ogni riga con transform
df["Media_Gruppo"] = df.groupby(["F1", "F2"])["Val"].transform("mean")

# 5. CONFRONTA con la media del gruppo
df["Delta"] = df["Val"] - df["Media_Gruppo"]
df["Sopra_Media"] = df["Delta"] > 0

# 6. IDENTIFICA anomalie
df["Anomalo"] = df["Delta"].abs() > df["Delta"].std() * 2
```

---

## 6.3 Applicazioni nel Mondo Reale

| Dominio | Applicazione |
|---------|--------------|
| **Retail** | Analisi vendite per (Categoria Prodotto × Stagione × Regione) |
| **Finance** | Confronto rendimenti per (Settore × Rating × Durata) |
| **Healthcare** | Outcome per (Trattamento × Gruppo Età × Sesso) |
| **Manufacturing** | Qualità per (Linea Produzione × Turno × Operatore) |

---

## 6.4 Collegamento alla Prossima Lezione

Nella **Lezione 5** inizieremo il percorso nel **Machine Learning**:
- Introduzione a Scikit-learn
- Train-test split
- Primo modello predittivo

---

# 7) Checklist di fine lezione

Prima di procedere alla prossima lezione, verifica di saper fare tutto quanto segue:

- [ ] So calcolare quantili con `.quantile(q)` dove q e tra 0 e 1
- [ ] Capisco la differenza tra Q1, Q2 (mediana), Q3
- [ ] So creare colonne booleane (flag) basate su condizioni
- [ ] So categorizzare variabili continue usando maschere e `.loc[]`
- [ ] So usare `.groupby()` per raggruppare dati
- [ ] So applicare `.groupby()` su piu colonne contemporaneamente
- [ ] Capisco il paradigma Split-Apply-Combine
- [ ] So la differenza tra `.mean()` (riduce) e `.transform("mean")` (mantiene righe)
- [ ] So usare `.transform()` per aggiungere statistiche di gruppo a ogni riga
- [ ] So calcolare quanto un valore e sopra/sotto la media del suo gruppo
- [ ] Ricordo di inizializzare le colonne prima di usare `.loc[]` per assegnazioni
- [ ] So che `.quantile(0.75)` e diverso da `.quantile(75)`

**Se hai dubbi su qualche punto, rileggi la sezione corrispondente prima di proseguire.**

---

# 📌 Sezione 8 – Changelog Didattico

## Storico delle modifiche

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | Originale | Versione iniziale del notebook |
| 2.0 | Ristrutturazione | Ristrutturazione completa secondo template a 8 sezioni |
| 2.1 | Espansione | Espansione massiva con spiegazioni dettagliate (1500+ char per passaggio) |

---

## Dettaglio modifiche versione 2.1

### Sezione 1 - Titolo e Obiettivi
- Aggiunta tabella prerequisiti con riferimenti
- Aggiunto "Perché questa lezione è cruciale?" con casi d'uso reali

### Sezione 2 - Teoria Profonda
- Espansa teoria quantili con tabella IQR
- Aggiunta tabella approcci categorizzazione
- Diagramma ASCII per groupby singolo vs multi

### Sezione 3 - Schema Mentale
- Aggiunti 4 diagrammi decisionali con box ASCII
- Aggiunta checklist pre-analisi
- Aggiunto workflow completo in 4 step

### Sezione 4 - Dimostrazione
- Tabella panoramica step della demo
- Spiegazioni "Perché?" prima di ogni cella
- Micro-checkpoint con assert in ogni cella
- Commenti dettagliati nel codice

### Sezione 5 - Reference e Esercizi
- Tabelle reference organizzate per categoria
- Glossario con 13 termini
- 7 errori comuni documentati con fix

### Sezione 6 - Conclusione
- Pattern ricorrente completo con 6 step
- Applicazioni nel mondo reale

---

## Note per l'istruttore

- **Tempo stimato**: 60-75 minuti per esecuzione completa
- **Prerequisiti**: Lezioni 01-03 (NumPy, Pandas filtering, GroupBy base)
- **Punti critici**: 
  - Inizializzare colonne prima di `.loc[]`
  - Differenza `.quantile(0.75)` vs `.quantile(75)`
  - Transform multi-gruppo

---

**Status: LESSON COMPLETED ✓**

**Fine della Lezione 4**